测试CSTR反应器中Ziegler-Natta催化聚合反应的模型求解

案例一: 浆液法生产聚丙烯

![avatar](1.png)

本案例Aspen文件为 “案例1: pp_cstr”

本案例只计算反应器出口，不涉及其他单元。反应器含有气液两相，需耦合相平衡计算。

In [51]:
# 导包
import numpy as np
from polymer_model.kinetics.ziegler_nat.cstr2 import *
from polymer_model.thermodynamics.args import args

In [52]:
# 设置参数
# 设置组分参数：C3H6、C3H8、H2、N2、PP、TiCl4、TEA、C3H6-R
components = {
    "C3H6": {"type": "conventional"},
    "C3H8": {"type": "conventional"},
    "H2": {"type": "conventional"},
    "N2": {"type": "conventional"},
    "PP": {"type": "polymer"},
    "TiCl4": {"type": "conventional"},
    "TEA": {"type": "conventional"},
    "C3H6-R": {"type": "segment"},
}

# 设置聚合物链段
segments = {"C3H6-R":{"type": "repeat"}}

# 设置催化剂
catalyst = {"TiCl4": {"type": "Z-N",      # 催化剂类型
                    "site_types_num":4,   # 位点数
                    "site_conc":0.3,      # 位点浓度, mol/kgcat
            }}

# 设置聚合物
polymers = {
    "segments": segments,
    "catalyst": catalyst,
}

In [53]:
# 设置进料参数: 温度、压力、摩尔流量、摩尔分数、催化剂各位点流量、入口聚合物各阶矩流量
feed = {
    "T": 333.1399,
    "P": 3e6,
    "Mole Flow": 7022.557,  
    "Mole Fraction": {
        "C3H6": 0.7707859,
        "C3H8": 0.1435218,
        "H2": 0.0784869,
        "N2": 0.00720121,
        "PP": 0,
        "TiCl4": 6.25572E-07,
        "TEA": 3.46468E-06,
    },
    "TiCl4": {
        "CPSFLOW":0.00025, 
        "CDSFLOW":0, 
        "CISFLOW":np.array([0, 0, 0, 0]), 
        "CVSFLOW":np.array([0, 0, 0, 0])
    },
    "PP": {
        "LSZMOM": np.array([0, 0, 0, 0]),
        "DSZMOM": np.array([0, 0, 0, 0]),
        "LSFMOM": np.array([0, 0, 0, 0]),
        "DSFMOM": np.array([0, 0, 0, 0]),
        "LSSMOM": np.array([0, 0, 0, 0]),
        "DSSMOM": np.array([0, 0, 0, 0]),
    }
}

In [54]:
# 设置Ziegler-Natta反应物种
species = { "polymer": "PP",          # 聚合物
            "tdb segment": None,      # 终端双键(用于计算支链)
            "monomers": "C3H6",       # 单体
            "segments": "C3H6-R",     # 链段
            "precatalyst": None,      # 预催化剂
            "catalyst": "TiCl4",      # 催化剂
            "cocatalysts": "TEA",     # 助催化剂
            "solvents": None,         # 溶剂
            "transfer agent": None,   # 链转移剂
            "hydrogens": "H2",        # 氢
            "poisons": None,          # 毒物
            "elec don": None,         # 电子供体
            "byproduct": None,        # 副产物
        }

In [55]:
# 设置Ziegler-Natta反应动力学参数
# 反应类型, 催化位点, 组分1, 组分2, 前指因子, 活化能, 反应级数, 终端双键分数, 参考温度
r1 = [["Act-Spon", 1, "TiCl4", None, 0.0013, 3.19872e4, 1, None, 343.15],
    ["Act-Spon", 2, "TiCl4", None, 0.0013, 3.19872e4, 1, None, 343.15],
    ["Act-Spon", 3, "TiCl4", None, 0.0013, 3.19872e4, 1, None, 343.15],
    ["Act-Spon", 4, "TiCl4", None, 0.0013, 3.19872e4, 1, None, 343.15],
    ["Chain-Ini", 1, "C3H6", None, 108.85/1000, 3.0145e4, 1, None, 343.15],
    ["Chain-Ini", 2, "C3H6", None, 24.5/1000,   3.0145e4, 1, None, 343.15],
    ["Chain-Ini", 3, "C3H6", None, 170.8/1000,  3.0145e4, 1, None, 343.15],
    ["Chain-Ini", 4, "C3H6", None, 60.55/1000,  3.0145e4, 1, None, 343.15],
    ["Propagation", 1, "C3H6-R", "C3H6", 108.85/1000,  3.0145e4, 1, None, 343.15],
    ["Propagation", 2, "C3H6-R", "C3H6", 24.5/1000,  3.0145e4, 1, None, 343.15],
    ["Propagation", 3, "C3H6-R", "C3H6", 170.8/1000,  3.0145e4, 1, None, 343.15],
    ["Propagation", 4, "C3H6-R", "C3H6", 60.55/1000,  3.0145e4, 1, None, 343.15],
    ["Chat-Mon", 1, "C3H6-R", "C3H6", 0.012/1000,  5.2e4, 1, None, 343.15],
    ["Chat-Mon", 2, "C3H6-R", "C3H6", 0.012/1000,  5.2e4, 1, None, 343.15],
    ["Chat-Mon", 3, "C3H6-R", "C3H6", 0.012/1000,  5.2e4, 1, None, 343.15],
    ["Chat-Mon", 4, "C3H6-R", "C3H6", 0.012/1000,  5.2e4, 1, None, 343.15],
    ["Chat-Cocat", 1, "C3H6-R", "TEA", 0.12/1000,  5.02416e4, 1, None, 343.15],
    ["Chat-Cocat", 2, "C3H6-R", "TEA", 0.12/1000,  5.02416e4, 1, None, 343.15],
    ["Chat-Cocat", 3, "C3H6-R", "TEA", 0.12/1000,  5.02416e4, 1, None, 343.15],
    ["Chat-Cocat", 4, "C3H6-R", "TEA", 0.12/1000,  5.02416e4, 1, None, 343.15],
    ["Chat-H2", 1, "C3H6-R", "H2", 4.8/1000,  4.47988e4, 1, None, 343.15],
    ["Chat-H2", 2, "C3H6-R", "H2", 8.88/1000,  4.47988e4, 1, None, 343.15],
    ["Chat-H2", 3, "C3H6-R", "H2", 2.64/1000,  4.47988e4, 1, None, 343.15],
    ["Chat-H2", 4, "C3H6-R", "H2", 6.6/1000,  4.47988e4, 1, None, 343.15],
    ["Deact-Spon", 1, None, None, 0.001,  4.1868e3, 1, None, 343.15],
    ["Deact-Spon", 2, None, None, 0.001,  4.1868e3, 1, None, 343.15],
    ["Deact-Spon", 3, None, None, 0.001,  4.1868e3, 1, None, 343.15],
    ["Deact-Spon", 4, None, None, 0.001,  4.1868e3, 1, None, 343.15],
    ["Atact-Prop", 1, "C3H6-R", "C3H6", 8/1000,  3.0145e4, 1, None, 343.15],
    ["Atact-Prop", 2, "C3H6-R", "C3H6", 1/1000,  3.0145e4, 1, None, 343.15],
    ["Atact-Prop", 3, "C3H6-R", "C3H6", 0.1/1000,  3.0145e4, 1, None, 343.15],
    ["Atact-Prop", 4, "C3H6-R", "C3H6", 0.1/1000,  3.0145e4, 1, None, 343.15]]

In [56]:
# 设置反应器参数: 反应温度、压力、有效相态、计算反应体积的参数(参考Aspen)
specs = {
    "P": 3e6,
    "T": 333.15,
    "Valid phases": "VL",
    "Specification type": "Reactor volume & Phase volume",
    "Reactor volume": 90,
    "Condensed phase volume": 60,
    "Reactions": {"Type":"Z-N", 
                    "Species":species, 
                    "Reactions":r1, 
                    "Reacting phase":"liquid"}
}

In [57]:
# 运行计算程序，输出出口流量、组成、各位点各阶矩
product = cstr(components,args, polymers, [feed], specs)
print(product)

{'T': 333.15, 'P': 3000000.0, 'Mole Flow': 7022.5497619267135, 'Mole Fraction': {'C3H6': 0.768254005045942, 'C3H8': 0.14352194792651413, 'H2': 0.07848600304178821, 'N2': 0.007201217422216645, 'PP': 0.0025327363467456754, 'TiCl4': 6.255726447709357e-07, 'TEA': 3.464644148552338e-06}, 'TiCl4': {'CPSFLOW': 0.00011693290385144051, 'CDSFLOW': 3.0168891898226477e-05, 'CISFLOW': array([0., 0., 0., 0.]), 'CVSFLOW': array([7.13730514e-09, 5.79389166e-08, 2.54848806e-09, 1.75261730e-08])}, 'PP': {'SFLOW': {'C3H6-R': 1.0}, 'SFRAC': {'C3H6-R': 1.0}, 'LSZMOM': array([2.57174138e-05, 2.56666121e-05, 2.57220026e-05, 2.57070249e-05]), 'DSZMOM': array([0.00191181, 0.00312907, 0.00126432, 0.00245058]), 'LSFMOM': array([0.07045996, 0.0097238 , 0.16607674, 0.030657  ]), 'DSFMOM': array([5.23792097, 1.18544792, 8.16320828, 2.92244264]), 'LSSMOM': array([ 386.0185618 ,    7.35799722, 2144.41675253,   73.08955833]), 'DSSMOM': array([ 28696.22410264,    897.02849093, 105405.01294071,   6967.41520568]), 'ZMOM'

In [58]:
# 计算链长分布
# n = get_points(Np=100, upper=100000)
# eqs, τ, SFMOM, mw = model(sol.x, components,args, ploymers, feed, CSTR) 
# w = cld(τ, SFMOM, n, GPC=True)
# print("局部分子量分布: \n", w)

In [59]:
# 绘制链长分布图
# cld_plot(n, w)

In [60]:
# 计算分子量分布
# Mn = n*mw
# w = mwd(τ, SFMOM, mw, Mn)
# print("局部分子量分布: \n", w)

In [61]:
# 绘制分子量分布图
# mwd_plot(Mn,w)